In [1]:
import os
import sys
from dotenv import load_dotenv
from crewai import Crew, Process, Task
from langchain_huggingface import HuggingFaceEndpoint

# Configurar path para importar desde src/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Cargar variables de entorno
load_dotenv('../.env')

# Verificar API Key
hf_token = os.getenv('HUGGING_FACE_HUB_TOKEN')
if not hf_token:
    raise ValueError(
        "HUGGING_FACE_HUB_TOKEN no encontrada.\n"
        "Asegúrate de que tu archivo .env está en la raíz del proyecto."
    )

print("✓ Configuración inicial completada")
print(f"✓ Token HF cargado (longitud: {len(hf_token)} caracteres)")

c:\Users\Usuario\anaconda3\envs\multiagent_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Configuración inicial completada
✓ Token HF cargado (longitud: 37 caracteres)


In [2]:
from src.agents import researcher, writer, reviewer, llm

print("\n✓ Agentes y LLM importados correctamente:")
print(f"  1. {researcher.role}")
print(f"  2. {writer.role}")
print(f"  3. {reviewer.role}")
print(f"\n✓ LLM importado: {llm.model}")

API Key cargada correctamente
✓ LLM (Cerebro) configurado: mistralai/Mistral-7B-Instruct-v0.3
Herramienta de búsqueda configurada
✓ Agentes definidos correctamente
  - Investigador Senior de IA
  - Escritor Técnico de IA
  - Editor Crítico de IA

✓ Agentes y LLM importados correctamente:
  1. Investigador Senior de IA
  2. Escritor Técnico de IA
  3. Editor Crítico de IA

✓ LLM importado: huggingface/meta-llama/Meta-Llama-3-8B-Instruct


In [3]:
# TAREA 1: Investigación (Researcher)
task_research = Task(
    description=(
        "Busca en la web información actualizada y relevante sobre 'Sesgo en Modelos "
        "de Lenguaje Grandes (LLMs)'. Enfócate en:\n"
        "1. Definiciones y tipos de sesgo (género, raza, cultural)\n"
        "2. Estudios recientes y ejemplos concretos\n"
        "3. Causas del sesgo (datos de entrenamiento, arquitectura)\n"
        "4. Impacto en aplicaciones reales\n\n"
        "Prioriza fuentes académicas, blogs técnicos reconocidos (Medium, Towards Data Science), "
        "y sitios oficiales de investigación en IA. Proporciona al menos 5 fuentes distintas "
        "con sus URLs."
    ),
    expected_output=(
        "Un reporte detallado con:\n"
        "- Lista de 5-7 fuentes confiables con títulos y URLs\n"
        "- Resumen de 2-3 oraciones de cada fuente\n"
        "- Temas clave identificados\n"
        "- Citas o datos relevantes extraídos"
    ),
    agent=researcher
)

# TAREA 2: Primer Borrador (Writer)
task_write_draft = Task(
    description=(
        "Usando ÚNICAMENTE la información recopilada por el Investigador, redacta un "
        "PRIMER BORRADOR de resumen de EXACTAMENTE 500 palabras en formato Markdown "
        "con la siguiente estructura:\n\n"
        "# Sesgo en Modelos de Lenguaje Grandes (LLMs)\n\n"
        "## Introducción\n"
        "(100 palabras: contexto, importancia del tema)\n\n"
        "## Hallazgos Clave\n"
        "(200 palabras: principales descubrimientos, ejemplos concretos)\n\n"
        "## Desafíos Éticos y Técnicos\n"
        "(150 palabras: problemas identificados, impacto)\n\n"
        "## Conclusión\n"
        "(50 palabras: reflexión final, perspectivas futuras)\n\n"
        "IMPORTANTE:\n"
        "- Usa un lenguaje técnico pero accesible\n"
        "- Incluye datos específicos o estadísticas cuando estén disponibles\n"
        "- Mantén objetividad y balance\n"
        "- Cita las fuentes entre paréntesis cuando sea relevante\n"
        "- Este es un BORRADOR, el Revisor lo evaluará después"
    ),
    expected_output=(
        "Un documento Markdown de 500 palabras (+/- 50) con las 4 secciones requeridas, "
        "bien estructurado, coherente y basado fielmente en la investigación proporcionada. "
        "Debe estar claramente marcado como 'BORRADOR'."
    ),
    agent=writer,
    context=[task_research]
)

# TAREA 3: Revisión Crítica (Reviewer)
task_review = Task(
    description=(
        "Revisa CRÍTICAMENTE el borrador generado por el Escritor. Evalúa:\n\n"
        "1. **Coherencia**: ¿Las ideas fluyen lógicamente?\n"
        "2. **Precisión Factual**: ¿La información coincide con las fuentes del Investigador?\n"
        "3. **Estructura**: ¿Se respeta el formato y extensión requeridos?\n"
        "4. **Claridad**: ¿Es comprensible para la audiencia objetivo?\n"
        "5. **Markdown**: ¿El formato está correcto?\n\n"
        "Proporciona un reporte detallado que incluya:\n"
        "- Calificación general (Aprobado/Necesita correcciones menores/Requiere reescritura)\n"
        "- Lista ESPECÍFICA de problemas encontrados con ubicación (sección/párrafo)\n"
        "- Sugerencias CONCRETAS de mejora para cada problema\n"
        "- Correcciones de errores factuales\n"
        "- Recomendaciones de contenido adicional si falta algo importante\n\n"
        "Sé constructivo pero riguroso. El Escritor usará tu feedback para la versión final."
    ),
    expected_output=(
        "Un reporte de revisión estructurado con:\n"
        "1. Resumen ejecutivo de la evaluación\n"
        "2. Lista numerada de issues con ubicación específica\n"
        "3. Sugerencias concretas de mejora\n"
        "4. Recomendaciones prioritarias para la versión final\n"
        "5. Aspectos positivos del borrador (qué mantener)"
    ),
    agent=reviewer,
    context=[task_research, task_write_draft]
)

# TAREA 4: Versión Final (Writer)
task_write_final = Task(
    description=(
        "Basándote en el feedback DETALLADO del Revisor, genera la VERSIÓN FINAL "
        "del resumen corrigiendo TODOS los problemas identificados:\n\n"
        "1. Lee cuidadosamente el reporte de revisión\n"
        "2. Implementa TODAS las correcciones sugeridas\n"
        "3. Ajusta contenido donde el Revisor indicó inconsistencias factuales\n"
        "4. Mejora la claridad en las secciones señaladas\n"
        "5. Asegura que la estructura y formato Markdown sean perfectos\n"
        "6. Verifica que sean exactamente 500 palabras (+/- 20)\n\n"
        "La versión final debe:\n"
        "- Mantener la estructura de 4 secciones\n"
        "- Estar libre de los errores identificados por el Revisor\n"
        "- Ser clara, precisa y bien escrita\n"
        "- Estar lista para publicación SIN más ediciones\n\n"
        "NO incluyas notas como 'VERSIÓN FINAL' o 'CORREGIDO', solo el contenido limpio."
    ),
    expected_output=(
        "El documento Markdown FINAL y PERFECCIONADO de 500 palabras, sin errores, "
        "con todas las correcciones del Revisor implementadas. Este es el documento "
        "que se guardará como research_summary.md."
    ),
    agent=writer,
    context=[task_research, task_write_draft, task_review]
)

print("\n✓ Tareas definidas:")
print("  1. Investigación (Researcher)")
print("  2. Primer Borrador (Writer)")
print("  3. Revisión Crítica (Reviewer)")
print("  4. Versión Final (Writer)")
print("\n Ciclo de comunicación:")
print("  Researcher → Writer (draft) → Reviewer → Writer (final)")


✓ Tareas definidas:
  1. Investigación (Researcher)
  2. Primer Borrador (Writer)
  3. Revisión Crítica (Reviewer)
  4. Versión Final (Writer)

 Ciclo de comunicación:
  Researcher → Writer (draft) → Reviewer → Writer (final)


In [4]:
crew = Crew(
    agents=[researcher, writer, reviewer],
    tasks=[task_research, task_write_draft, task_review, task_write_final],
    process=Process.sequential,
    verbose=True,
)

print("\n✓ Crew creado exitosamente")
print("  Proceso: Secuencial (4 tareas)")
print("  Agentes: 3 (Researcher, Writer, Reviewer)")
print("  Tareas: 4 (Research → Draft → Review → Final)")


✓ Crew creado exitosamente
  Proceso: Secuencial (4 tareas)
  Agentes: 3 (Researcher, Writer, Reviewer)
  Tareas: 4 (Research → Draft → Review → Final)


In [5]:
print("\n" + "="*70)
print("INICIANDO WORKFLOW DE INVESTIGACIÓN COLABORATIVA")
print("="*70 + "\n")
print("Este proceso puede tomar 8-15 minutos...")
print("\n Ciclo de trabajo:")
print("  ┌─ 1. Researcher busca información en la web")
print("  ├─ 2. Writer genera el primer borrador")
print("  ├─ 3. Reviewer evalúa y critica el borrador")
print("  └─ 4. Writer produce la versión final corregida\n")
print(" Iniciando ejecución...\n")

# Ejecutar el crew
result = crew.kickoff()

print("\n" + "="*70)
print(" WORKFLOW COMPLETADO EXITOSAMENTE")
print("="*70 + "\n")


INICIANDO WORKFLOW DE INVESTIGACIÓN COLABORATIVA

Este proceso puede tomar 8-15 minutos...

 Ciclo de trabajo:
  ┌─ 1. Researcher busca información en la web
  ├─ 2. Writer genera el primer borrador
  ├─ 3. Reviewer evalúa y critica el borrador
  └─ 4. Writer produce la versión final corregida

 Iniciando ejecución...



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d6a00ac0-7174-431c-8653-b310970e82e5                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Investigador Senior de IA                                                                               │
│                                                                                                                 │
│  Task: Busca en la web información actualizada y relevante sobre 'Sesgo en Modelos de Lenguaje Grandes          │
│  (LLMs)'. Enfócate en:                                                                                          │
│  1. Definiciones y tipos de sesgo (género, raza, cultural)                                                      │
│  2. Estudios recientes y ejemplos concretos                                                                     │
│  3. Causas del sesgo (datos de entrenamiento, arquitectura)                                                     │
│  4. Impacto en aplicaciones reales                                                                              │
│                                                                                                                 │
│  Prioriza fuentes académicas, blogs técnicos reconocidos (Medium, Towards Data Science), y sitios oficiales de  │
│  investigación en IA. Proporciona al menos 5 fuentes distintas con sus URLs.                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\Usuario\anaconda3\envs\multiagent_2\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

c:\Users\Usuario\anaconda3\envs\multiagent_2\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

c:\Users\Usuario\anaconda3\envs\multiagent_2\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 11b7732f-0e9b-40b2-bbfb-8032965c2db8                                                                     │
│  Agent: Escritor Técnico de IA                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d6a00ac0-7174-431c-8653-b310970e82e5                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Sesgo en Modelos de Lenguaje Grandes (LLMs)                                                    │
│                                                                                                                 │
│  ## Introducción                                                                                                │
│                                                                                                                 │
│  Los Modelos de Lenguaje Grandes (LLMs) han revolucionado la forma en que interactuamos con la tecnología,      │
│  pero también han planteado serias preocupaciones sobre el sesgo y la discriminación. La falta de               │
│  representación equitativa en los conjuntos de datos de entrenamiento, la presencia de sesgos en la             │
│  arquitectura y el diseño de los modelos, y la influencia de los prejuicios presentes en la sociedad humana     │
│  son algunas de las causas clave del sesgo en LLMs (OpenAI, 2025; Fundar, 2025). Es importante que los          │
│  desarrolladores, usuarios y la sociedad en general se involucren en la identificación y mitigación del sesgo   │
│  en LLMs para crear tecnologías más justas y equitativas.                                                       │
│                                                                                                                 │
│  ## Causas del Sesgo en LLMs                                                                                    │
│                                                                                                                 │
│  El sesgo en LLMs se debe a varias causas, incluyendo:                                                          │
│                                                                                                                 │
│  *   La falta de representación equitativa en los conjuntos de datos de entrenamiento.                          │
│  *   La presencia de sesgos en la arquitectura y el diseño de los modelos.                                      │
│  *   La influencia de los prejuicios presentes en la sociedad humana.                                           │
│                                                                                                                 │
│  ## Consecuencias del Sesgo en LLMs                                                                             │
│                                                                                                                 │
│  El sesgo en LLMs puede tener graves consecuencias, incluyendo la perpetuación de estereotipos y prejuicios,    │
│  la discriminación y la exclusión de grupos marginados, y la pérdida de confianza en la tecnología y la         │
│  sociedad (Wikipedia, 2023; Skillnest, 2025).                                                                   │
│                                                                                                                 │
│  ## Estrategias para Mitigar el Sesgo en LLMs                                                                   │
│                                                                                                                 │
│  Para mitigar el sesgo en LLMs, se pueden implementar 


 WORKFLOW COMPLETADO EXITOSAMENTE



┌───────────────────────── Trace Batch Finalization ──────────────────────────┐
│ ✅ Trace batch finalized with session ID:                                   │
│ 3aa53c1c-f588-4fc2-8dd5-5224819cace2                                        │
│                                                                             │
│ 🔗 View here:                                                               │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/3aa53c1c-f588-4f │
│ c2-8dd5-5224819cace2?access_code=TRACE-c37c23eec2                           │
│ 🔑 Access Code: TRACE-c37c23eec2                                            │
└─────────────────────────────────────────────────────────────────────────────┘


In [8]:
# Crear directorio data si no existe
data_dir = '../data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"✓ Directorio creado: {data_dir}")

# Extraer el contenido de texto del CrewOutput
result_text = result.raw  

# Guardar el resultado en data/research_summary.md
output_path = os.path.join(data_dir, 'research_summary.md')

with open(output_path, 'w', encoding='utf-8') as f:
    f.write(result_text)

print(f"\n✓ Resumen final guardado en: {output_path}")
print(f"✓ Longitud del documento: {len(result_text)} caracteres")
print(f"✓ Palabras aproximadas: {len(result_text.split())} palabras")


✓ Resumen final guardado en: ../data\research_summary.md
✓ Longitud del documento: 3054 caracteres
✓ Palabras aproximadas: 474 palabras
